In [1]:
from pyppeteer import launch
import asyncio
import nest_asyncio
import OnlineMatch.global_data as gb
import OnlineMatch.monitor_result as mon

nest_asyncio.apply()

'https://dkmgames.com/Gomoku/gomoku.htm'

In [2]:
async def start_computer_game():
    """
    Go to the match ready page
    :return: None
    """
    mon.print_time_and_msg("Setting up browser for web page...")
    url = "https://dkmgames.com/Gomoku/gomoku.htm"

    gb.browser = await launch({
        "headless": False,
        "autoClose": False
    })
    page = await gb.browser.newPage()

    await page.setViewport({'width': 1920, 'height': 1080})
    await page.goto(url, {'timeout': 10000 * 30})

    while not await page.J("#newgame"):
        mon.random_wait()
    mon.print_time_and_msg(f"Website is rendered!")

    # Select opponent
    # await page.click(gb.opponent_selector)
    # mon.print_time_and_msg(f"Selected opponent as {gb.opponent_choice}")
    # mon.random_wait()

    # Select color
    await page.click(gb.color_selector)
    mon.print_time_and_msg(f"Color selected as {gb.start_color}")
    mon.random_wait()

    # Select level
    await page.click(gb.level_selector)
    mon.print_time_and_msg(f"Selected level {gb.level_choice}")
    mon.random_wait()

    # Uncheck 4 line warning
    await page.click("#cbWarn")
    mon.random_wait()

    # Click start
    start_btn_selector = "#btnPlay"
    await page.click(start_btn_selector)
    mon.print_time_and_msg(f"Clicked start button ")
    mon.random_wait()

    gb.page = page
    asyncio.get_event_loop().run_until_complete(get_img_position())


In [3]:
async def get_img_position():
    """
    Get the chessboard location
    :return: Location tuple
    """

    board_selector = "#grid"
    chessboard = await gb.page.J(board_selector)
    location = await chessboard.boundingBox()  # location is a dict
    mon.print_time_and_msg(f"Got chessboard location: {location}")
    gb.board_start_x = location['x']
    gb.board_start_y = location['y']
    gb.board_location = location


async def get_chessboard_img(who: str):
    """
    Get the screenshot of the chessboard
    :param: Who moved
    :return: None
    """
    captcha_option = {
        "path": f"Pictures/chessboard-{who}.png",
        "type": "png",
        "clip": gb.board_location
    }
    await gb.page.screenshot(captcha_option)
    mon.print_time_and_msg(f"Took screenshot chessboard-{who}.png")

In [4]:
async def move():
    place_x = gb.board_start_x + 38.13333 * 2
    place_y = gb.board_start_y + 38.13333 * 2
    await gb.page.mouse.click(place_x, place_y)
    asyncio.get_event_loop().run_until_complete(
        get_chessboard_img("some")
    )

In [7]:
asyncio.get_event_loop().run_until_complete(
        start_computer_game()
)
asyncio.get_event_loop().run_until_complete(
    get_chessboard_img("opp")
)


[02:54:00:11] - Setting up browser for web page...
[02:54:04:11] - Website is rendered!
[02:54:04:11] - Color selected as w
[02:54:05:11] - Selected level 3
[02:54:06:11] - Clicked start button 
[02:54:07:11] - Got chessboard location: {'x': 20, 'y': 10, 'width': 572, 'height': 572}
[02:54:07:11] - Took screenshot chessboard-opp.png


In [6]:
asyncio.get_event_loop().run_until_complete(
    move()
)



[02:45:50:11] - Took screenshot chessboard-some.png


In [8]:
func = "function sleep(milliseconds) {\
  var start = new Date().getTime();\
  for (var i = 0; i < 1e7; i++) {\
    if ((new Date().getTime() - start) > milliseconds){\
      break;\
    }\
  }\
}"

In [11]:
gb.page.evaluate(pageFunction=func)

<coroutine object Page.evaluate at 0x000002753E70FE48>

In [12]:
gb.page.evaluate(pageFunction="sleep(5000)")



<coroutine object Page.evaluate at 0x000002753E70F948>